# Introduction to Argovis's API

Argovis provides an API that indexes and distributes numerous oceanographic datasets with detailed query parameters, enabling you to search and download only and exactly data of interest. In this notebook, we'll tour some of the standard usage patterns enabled by Argovis.

## Setup: Register an API key

In order to allocate Argovis's limited computing resources fairly, users are encouraged to register and request a free API key. This works like a password that identifies your requests to Argovis. To do so:

 - Visit [https://argovis-keygen.colorado.edu/](https://argovis-keygen.colorado.edu/)
 - Fill out the form under _New Account Registration_
 - An API key will be emailed to you shortly.
 
Treat this API key like a password - don't share it or leave it anywhere public. If you ever forget it or accidentally reveal it to a third party, see the same website above to change or deactivate your token.

Put your API key in the quotes in the variable below before moving on:

In [1]:
API_ROOT='https://argovis-api.colorado.edu/'
API_KEY=''

# Argovis data structures

Argovis standard data structures divide measurements into _data_ and _metadata_ documents. Typically, a data document corresponds to measurements or gridded data associated with a discreet temporospatial column - a time, latitude and longitude. A single such document may contain measurements at multiple depths or altitudes, provided they share the same latitude, longitude, and time.

Each of these data documents will refer to at least one corresponding metadata document that captures additional information about the measurement. Argovis divides information between data and metadata documents in order to minimize redundancy in the data you download: many data documents will point to the same metadata document, allowing you to only download that metadata once. Typically, these metadata groupings will refer to some meaningful characteristic of the data; Argo metadata documents correspond to physical floats, while CCHDO metadata documents correspond to cruises, for example.

For more detail and specifications on the data and metadata documents for each collection, see [https://argovis.colorado.edu/docs/documentation/_build/html/data_management/schema.html](https://argovis.colorado.edu/docs/documentation/_build/html/data_management/schema.html).

# The standard data routes

## What datasets does Argovis index?

Argovis supports several different data sets with the API and data structures described here. They and their corresponding routes are:

 - Argo profiling float data, `/argo`
 - CCHDO ship-based profile data, `/cchdo`
 - tropical cyclone data from HURDAT and JTWC, `/tc`
 - Global Drifter Program data, `/drifters`
 - several gridded products:
   - Roemmich-Gilson total temperature and salinity, `/grids/rg09`
   - ocean heat content, `/grids/kg21`
 - Argone Argo float position forecast model data, `/argone`
   
The examples that follow apply equally to all these routes; they all support similar query options and follow similar behavior patterns.

## Using Swagger and the `argovisHelpers` package to download data

In order to successfully explore Argovis data, there are two important tools to introduce in this section: Swagger, our API documentation engine, and `argovisHelpers`, our Python package of fuctions to help you access and interpret Argovis data.

### Using Swagger docs

Argovis' API documentation is found at [https://argovis-api.colorado.edu/docs/](https://argovis-api.colorado.edu/docs/). These docs are split into several categories; what follows applies to all categories _not_ marked experimental; the experimental categories are under development and may change or be removed at any time.

Categories have three typical routes:
 - The main _data route_, like `/argo`, or `/cchdo`. These routes provide the data documents for the dataset named in the route.
 - The _metadata route_, like `/argo/meta`. These routes provide the metadata documents referred to by data documents.
 - The _vocabulary route_, like `/argo/vocabulary`. These routes provide lists of possible options for search parameters used in the corresponding data and metadata routes.
 
Click on any of the routes, like `/argo` - a list of possible query string parameters are presented, with a short explanation of what they mean.

If you're familiar with REST APIs, this is enough information for you to construct a query string and issue a request in any programming environment that can facilitate an HTTP GET request. If you're working in Python, we provide a helper library, `argovisHelpers`, to manage these requests for you. Let's try it out by making our first request for Argo data, for profiles found within 100 km of a point in the South Atlantic in May 2011 (users of Python's `requests` module will notice a familiar pattern, providing the query string parameters listed in the Swagger docs and associated values as a dictionary):

In [2]:
from argovisHelpers import helpers as avh

In [3]:
argoSearch = {
    'startDate': '2011-05-01T00:00:00Z',
    'endDate': '2011-06-01T00:00:00Z',
    'center': '-22.5,0',
    'radius': 100
}

argoProfiles = avh.query('argo', options=argoSearch, apikey=API_KEY, apiroot=API_ROOT)

Let's have a look at what we get from the first profile returned:

In [4]:
argoProfiles[0]

{'_id': '4901283_003',
 'geolocation': {'type': 'Point', 'coordinates': [-23.139, -0.154]},
 'basin': 1,
 'timestamp': '2011-05-02T08:26:28.000Z',
 'date_updated_argovis': '2023-01-28T01:13:18.370Z',
 'source': [{'source': ['argo_bgc'],
   'url': 'ftp://ftp.ifremer.fr/ifremer/argo/dac/aoml/4901283/profiles/SD4901283_003.nc',
   'date_updated': '2022-06-29T21:21:10.000Z'},
  {'source': ['argo_core'],
   'url': 'ftp://ftp.ifremer.fr/ifremer/argo/dac/aoml/4901283/profiles/D4901283_003.nc',
   'date_updated': '2018-10-03T14:45:37.000Z'}],
 'cycle_number': 3,
 'geolocation_argoqc': 1,
 'profile_direction': 'A',
 'timestamp_argoqc': 1,
 'vertical_sampling_scheme': 'Primary sampling: averaged []',
 'data_info': [['doxy',
   'doxy_argoqc',
   'pressure',
   'pressure_argoqc',
   'salinity',
   'salinity_argoqc',
   'salinity_sfile',
   'salinity_sfile_argoqc',
   'temperature',
   'temperature_argoqc',
   'temperature_sfile',
   'temperature_sfile_argoqc'],
  ['units', 'data_keys_mode'],
  [['

This is a data document for Argo, matching the specification at [https://argovis.colorado.edu/docs/documentation/_build/html/data_management/schema.html](https://argovis.colorado.edu/docs/documentation/_build/html/data_management/schema.html). It contains the `timestamp` and `geolocation` properties that place this profile geospatially, and other parameters that typically change from point to point.

All data documents bear a `metadata` key, which is a pointer to the appropriate metadata document to find out more about this measurement. Let's fetch that document for this first profile by querying the `argo/meta` route for a doument with an `id` that matches this `metadata` pointer:

In [5]:
metaOptions = {
    'id': argoProfiles[0]['metadata'][0]
}

argoMeta = avh.query('argo/meta', options=metaOptions, apikey=API_KEY, apiroot=API_ROOT)
argoMeta

[{'_id': '4901283_m0',
  'data_type': 'oceanicProfile',
  'data_center': 'AO',
  'instrument': 'profiling_float',
  'pi_name': ['BRECK OWENS'],
  'platform': '4901283',
  'platform_type': 'SOLO_W',
  'fleetmonitoring': 'https://fleetmonitoring.euro-argo.eu/float/4901283',
  'oceanops': 'https://www.ocean-ops.org/board/wa/Platform?ref=4901283',
  'positioning_system': 'GPS',
  'wmo_inst_type': '851'}]

In addition to temporospatial searches, data and metadata routes typically support _category searches_, which are searches for documents that belong to certain categories. Which categories are available to search by changes logically from dataset to dataset; Argo floats can be searched by platform number, for example, while tropical cyclones can be searched by storm name. See the swagger docs for the full set of possibilities for each category; let's now use argo's platform category search to get all profiles collected by the same platform as the first profile above:

In [6]:
platformSearch = {
    'platform': argoMeta[0]['platform']
}

platformProfiles = avh.query('argo', options=platformSearch, apikey=API_KEY, apiroot=API_ROOT)
print(len(platformProfiles))

125


At the time of writing, 125 profiles are found for this platform in this way.

For all category searches, we may wish to know the full list of all possible values a category can take on; for this, there are the _vocabulary_ routes. Let's get a list of all possible Argo platforms we can search by:

In [7]:
platformVocabSearch = {
    'parameter': 'platform'
}

platforms = avh.query('argo/vocabulary', options=platformVocabSearch, apikey=API_KEY, apiroot=API_ROOT)
print(platforms[0:10])

['13857', '13858', '13859', '15819', '15820', '15851', '15852', '15853', '15854', '15855']


Here we just print out the first 10 platform IDs found, but all 17 thousand or so are present.

## Using the `data` query option

The astute reader may have noticed something about the data document shown above: there's no actual measurements included in it! By default, only the non-measurement data is returned, in order to minimize bandwidth consumed; in order to get back actual measurements and their QC flags, we must query and filter including the `data` parameter, the behavior of which we'll see in this section.

### Basic data request

Let's start by asking for one particular profile by ID, and ask for some temperature data to go with:

In [8]:
dataQuery = {
    'id': '4901283_003',
    'data': 'temperature'
}

profile = avh.query('argo', options=dataQuery, apikey=API_KEY, apiroot=API_ROOT)
print(profile[0]['data'])

[[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180, 182, 184, 186, 188, 190, 192, 194, 196, 198, 200, 202, 204, 206, 208, 210, 212, 214, 216, 218, 220, 222, 224, 226, 228, 230, 232, 234, 236, 238, 240, 242, 244, 246, 248, 250, 252, 254, 256, 258, 260, 262, 264, 266, 268, 270, 272, 274, 276, 278, 280, 282, 284, 286, 288, 290, 292, 294, 296, 298, 300, 302, 304, 306, 308, 310, 312, 314, 316, 318, 320, 322, 324, 326, 328, 330, 332, 334, 336, 338, 340, 342, 344, 346, 348, 350, 352, 354, 356, 358, 360, 362, 364, 366, 368, 370, 372, 374, 376, 378, 380, 382, 384, 386, 388, 390, 392, 394, 396, 398, 400, 402, 404, 406, 408, 410, 412, 414, 416, 418, 420, 4

The `data` key contains a list of lists of measurements. To interpret them, we use the `data_info` key:

In [10]:
print(profile[0]['data_info'])

[['pressure', 'temperature'], ['units', 'data_keys_mode'], [['decibar', 'D'], ['degree_Celsius', 'D']]]


`data_info` is always a list that contains exactly three items:

 - `data_info[0]` is the list of measurements returned in our `data` object, in the same order as `data`. So in the example above, `data[0]` are pressure measurements, while `data[1]` are temperature measurements. Note we got back pressures even though we only asked for temperatures; pressures are always provided where available as they are needed to meaningfully interpret all other data variables.
 - `data_info[1]` is a list of per-measurement variables. In the example above, pressure and temperature both have a `units` and a `data_keys_mode` associated with them.
 - `data_info[2]` is a rank 2 matrix with rows labeled by `data_info[0]` and columns by `data_info[1]`. So for the example above, this matrix indicates pressure has units 'decibar', and temperature has `data_keys_mode` 'D'.
 
With this information, we now understand how to interpret the `data` key above: the first list is a list of pressures measured in decibar, and the second list are corresponding temperature measurements measured in degrees C. Note that the ith elements in the data lists all correspond to the same level - in other words, `data[0][i],  data[1][i], data[2][1], ....` are all measurements corresponding to the ith level of this object.
 
> **Data and metadata precedence**: sometimes, you might see a given key on *both* a data document and its corresponding metadata document; when this happens, the value on the data document always takes precedence. `data_info` is a common example of this, which we'll see again below.

### Data inflation

If you find this format difficult to consume, another option is to use the `data_inflate` function from the argovis helpers package. This function will turn your data array into a list of dictionaries, one dictionary per level, with keys corresponding to the data values:

In [12]:
inflated_data = avh.data_inflate(profile[0])
inflated_data[0:10]

[{'pressure': 2, 'temperature': 28.669001},
 {'pressure': 4, 'temperature': 28.667999},
 {'pressure': 6, 'temperature': 28.722},
 {'pressure': 8, 'temperature': 28.816},
 {'pressure': 10, 'temperature': 28.823},
 {'pressure': 12, 'temperature': 28.826},
 {'pressure': 14, 'temperature': 28.830999},
 {'pressure': 16, 'temperature': 28.783001},
 {'pressure': 18, 'temperature': 28.775999},
 {'pressure': 20, 'temperature': 28.740999}]

This format is inefficient to download, but easy to read and work with. Long-time users of previous versions of Argovis may recognize this as similar to the legacy format of some of our data.

### Getting absolutely everything

What we've seen above allows us to be very targeted in the data we download; rather than being forced to spend time and bandwidth downloading data we aren't interested in, we can focus on just what we need. On the other hand, somtimes we really do want everything, and for that there's `data=all`:

In [13]:
dataQuery = {
    'id': '4901283_003',
    'data': 'all'
}

profile = avh.query('argo', options=dataQuery, apikey=API_KEY, apiroot=API_ROOT)
avh.data_inflate(profile[0])[0:10]

[{'doxy': None,
  'doxy_argoqc': 4,
  'pressure': 2,
  'pressure_argoqc': 1,
  'salinity': 35.574966,
  'salinity_argoqc': 1,
  'salinity_sfile': 35.574966,
  'salinity_sfile_argoqc': 1,
  'temperature': 28.669001,
  'temperature_argoqc': 1,
  'temperature_sfile': 28.669001,
  'temperature_sfile_argoqc': 1},
 {'doxy': None,
  'doxy_argoqc': 4,
  'pressure': 4,
  'pressure_argoqc': 1,
  'salinity': 35.573761,
  'salinity_argoqc': 1,
  'salinity_sfile': 35.573761,
  'salinity_sfile_argoqc': 1,
  'temperature': 28.667999,
  'temperature_argoqc': 1,
  'temperature_sfile': 28.667999,
  'temperature_sfile_argoqc': 1},
 {'doxy': None,
  'doxy_argoqc': 4,
  'pressure': 6,
  'pressure_argoqc': 1,
  'salinity': 35.626602,
  'salinity_argoqc': 1,
  'salinity_sfile': 35.626602,
  'salinity_sfile_argoqc': 1,
  'temperature': 28.722,
  'temperature_argoqc': 1,
  'temperature_sfile': 28.722,
  'temperature_sfile_argoqc': 1},
 {'doxy': None,
  'doxy_argoqc': 4,
  'pressure': 8,
  'pressure_argoqc': 1,

### Filtering behavior of data requests

Note that adding a specific data filter is a _firm requirement_ that all returned profiles have some meaningful data for _all_ variables listed. Try demanding chlorophyl-a in addition to temperature for our current profile of interest:

In [16]:
dataQuery = {
    'id': '4901283_003',
    'data': 'temperature,chla'
}

profile = avh.query('argo', options=dataQuery, apikey=API_KEY, apiroot=API_ROOT)
print(profile)

[]


We get nothing in our array of profiles; even though we asked for profile id '4901283_003' and we know it exists, `data=temperature,chla` filters our query down to _only_ profiles that have both temperature and chla reported; since the profile requested doesn't have any chla measurements, it is dropped from the returns in this case. This is useful if you only want to download profiles that definitely have data of interest; for example, try the same thing on our regional search from above:

In [17]:
argoSearch = {
    'startDate': '2011-05-01T00:00:00Z',
    'endDate': '2011-06-01T00:00:00Z',
    'center': '-22.5,0',
    'radius': 100,
    'data': 'temperature,chla'
}

argoProfiles = avh.query('argo', options=argoSearch, apikey=API_KEY, apiroot=API_ROOT)
print(len(argoProfiles))

0


Evidently Argo made no chlorophyl-a measurements in May 2011 within 100 km of our point of interest - a fact which we found using the data api without having to download or reduce any data at all. One final point on data filtering in this manner: it's not enough for a profile to nominally have a variable defined for it; it must have at least one non-null value reported for that variable somewhere in the search results. For example, when we did `data=all` for our profile of interest above, we saw dissolved oxygen, `doxy`, was defined for it. But:

In [19]:
dataQuery = {
    'id': '4901283_003',
    'data': 'doxy'
}

profile = avh.query('argo', options=dataQuery, apikey=API_KEY, apiroot=API_ROOT)
print(profile)

[]


Again our search is filtered down to nothing, since every level in that profile reported `None` for `doxy`.

### Search negation

Let's find some profiles that do actually have dissolved oxygen in them, this time with a slightly different geography search: let's look for everything in August 2017 within a polygon region, defined as a list of `[longitude, latitude]` points: 

In [1]:
dataQuery = {
    'startDate': '2017-08-01T00:00:00Z',
    'endDate': '2017-09-01T00:00:00Z',
    'polygon': [[-150,-30],[-155,-30],[-155,-35],[-150,-35],[-150,-30]],
    'data': 'doxy'
}

profiles = avh.query('argo', options=dataQuery, apikey=API_KEY, apiroot=API_ROOT)

NameError: name 'avh' is not defined

We find one profile with meaningful dissolved oxygen data in the region of interest.

The `data` key also accepts _tilde negation_, meaning 'filter for profiles that _don't_ contain this data', for example:

In [21]:
dataQuery = {
    'startDate': '2017-08-01T00:00:00Z',
    'endDate': '2017-09-01T00:00:00Z',
    'polygon': [[-150,-30],[-155,-30],[-155,-35],[-150,-35],[-150,-30]],
    'data': 'temperature,~doxy'
}

profiles = avh.query('argo', options=dataQuery, apikey=API_KEY, apiroot=API_ROOT)
print(len(profiles))

19


We get a collection of profiles that appear in the region of interest, and have temperature but _not_ dissolved oxygen. In this way, we can split up our downloads into groups of related and interesting profiles without re-downloading the same profiles over and over.

### Minimal data responses

Sometimes, we might want to use the `data` filter as we've seen to confine our attention to only profiles that have data of interest, but we're only interested in general or metadata about those measurements, and don't want to download the actual measurements; for this, we can add the `except-data-values` token:

In [22]:
dataQuery = {
    'startDate': '2017-08-01T00:00:00Z',
    'endDate': '2017-09-01T00:00:00Z',
    'polygon': [[-150,-30],[-155,-30],[-155,-35],[-150,-35],[-150,-30]],
    'data': 'doxy,except-data-values'
}

profiles = avh.query('argo', options=dataQuery, apikey=API_KEY, apiroot=API_ROOT)
print(profiles)

[{'_id': '5905107_001', 'geolocation': {'type': 'Point', 'coordinates': [-154.974, -32.415]}, 'basin': 2, 'timestamp': '2017-08-11T11:57:19.001Z', 'date_updated_argovis': '2023-01-27T08:37:16.397Z', 'source': [{'source': ['argo_bgc'], 'url': 'ftp://ftp.ifremer.fr/ifremer/argo/dac/aoml/5905107/profiles/SD5905107_001.nc', 'date_updated': '2022-07-09T07:14:33.000Z'}, {'source': ['argo_core'], 'url': 'ftp://ftp.ifremer.fr/ifremer/argo/dac/aoml/5905107/profiles/D5905107_001.nc', 'date_updated': '2019-06-24T15:29:23.000Z'}], 'cycle_number': 1, 'geolocation_argoqc': 1, 'profile_direction': 'A', 'timestamp_argoqc': 1, 'vertical_sampling_scheme': 'Primary sampling: mixed [deeper than nominal 985dbar: discrete; nominal 985dbar to surface: 2dbar-bin averaged]', 'data_info': [['doxy', 'pressure'], ['units', 'data_keys_mode'], [['micromole/kg', 'D'], ['decibar', 'D']]], 'metadata': ['5905107_m0']}]


Note that specifying only `'data': 'except-data-values'` is the same as just leaving the `data` query key off completely; the purpose of this option is to allow you to filter by data, but then only get back the lightweight non-measurement values. 

If we want an even more minimal response, we can use the `compression=minimal` option:

In [23]:
dataQuery = {
    'startDate': '2017-08-01T00:00:00Z',
    'endDate': '2017-09-01T00:00:00Z',
    'polygon': [[-150,-30],[-155,-30],[-155,-35],[-150,-35],[-150,-30]],
    'data': 'doxy',
    'compression': 'minimal'
}

profiles = avh.query('argo', options=dataQuery, apikey=API_KEY, apiroot=API_ROOT)
print(profiles)

[['5905107_001', -154.974, -32.415, '2017-08-11T11:57:19.001Z', ['argo_bgc', 'argo_core']]]


With `compression: minimal`, for each data document we get only a minimal amount of information describing it; each data product has a slightly different minimal representation tailored to suit.